## You're here! 
Welcome to your first competition in the [ITI's AI Pro training program](https://ai.iti.gov.eg/epita/ai-engineer/)! We hope you enjoy and learn as much as we did prepairing this competition.


## Introduction

In the competition, it's required to predict the `Severity` of a car crash given info about the crash, e.g., location.

This is the getting started notebook. Things are kept simple so that it's easier to understand the steps and modify it.

Feel free to `Fork` this notebook and share it with your modifications **OR** use it to create your submissions.

### Prerequisites
You should know how to use python and a little bit of Machine Learning. You can apply the techniques you learned in the training program and submit the new solutions! 

### Checklist
You can participate in this competition the way you perefer. However, I recommend following these steps if this is your first time joining a competition on Kaggle.

* Fork this notebook and run the cells in order.
* Submit this solution.
* Make changes to the data processing step as you see fit.
* Submit the new solutions.

*You can submit up to 5 submissions per day. You can select only one of the submission you make to be considered in the final ranking.*


Don't hesitate to leave a comment or contact me if you have any question!

## Import the libraries

We'll use `pandas` to load and manipulate the data. Other libraries will be imported in the relevant sections.

In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.impute import KNNImputer


## Exploratory Data Analysis
In this step, one should load the data and analyze it. However, I'll load the data and do minimal analysis. You are encouraged to do thorough analysis!

Let's load the data using `pandas` and have a look at the generated `DataFrame`.

In [2]:
dataset_path = '/kaggle/input/car-crashes-severity-prediction/'

df = pd.read_csv(os.path.join(dataset_path, 'train.csv'))

wdf = pd.read_csv(os.path.join(dataset_path, 'weather-sfcsv.csv'))
# print(type(str(wdf["Year"])))
# wdf["date"] = str(wdf["Year"])+"-"+str(wdf["Day"])+"-"+str(wdf["Month"])
wdf['Hour'] = wdf['Hour'].map(int).apply(lambda x: "{:02d}".format(x))
wdf['timestamp'] = wdf["Year"].map(str) +'-'+wdf["Month"].map(str)+'-'+ wdf["Day"].map(str)+" "+wdf['Hour'].map(str)
print("The shape of the dataset is {}.\n\n".format(df.shape))




df.head()
wdf.head()

# wdf[wdf['timestamp']=='2019-6-12 08']
# wdf[wdf['timestamp'].duplicated()  ]and wdf['Wind_Chill(F)'].isna
# kdf = wdf[wdf['timestamp'].duplicated(keep='first')]
# kdf['timestamp'].value_counts()

# wdf[wdf['timestamp']=='2019-12-7 15']

# kdf['Wind_Chill(F)'].isna().sum()

wdf =wdf.drop(columns=['Year','Month','Hour','Day','Precipitation(in)','Selected'])
# wdf['Weather_Condition']= wdf['Weather_Condition'].astype('category')
# wdf['Weather_Condition'] = wdf['Weather_Condition'].cat.codes


# wdf.dropna(subset=['Visibility(mi)'], how='all', inplace=True)
# wdf.dropna(subset=['Weather_Condition'], how='all', inplace=True)
wdf['Weather_Condition'] = wdf['Weather_Condition'].map({'Partly Cloudy': 0, 'Mostly Cloudy': 1, 'Fair': 2, 'Overcast': 3, 'Clear': 4, 'Scattered Clouds': 5,'Cloudy': 6, 'Light Rain': 7, 'Fair / Windy': 8, 'Haze': 9, 'Rain': 10, 'Partly Cloudy / Windy': 11, 'Mostly Cloudy / Windy': 12, 'Fog': 13, 'Shallow Fog': 14,'Heavy Rain': 15, 'Smoke': 16, 'Cloudy / Windy': 17, 'Light Rain / Windy':18 , 'Light Drizzle': 19, 'Mist': 20, 'Patches of Fog': 21, 'Rain / Windy': 22, 'Light Thunderstorms and Rain': 23, 'Squalls': 24,'Fog / Windy': 25})


# wdf['Weather_Condition'].value_counts()


# integrate missing data in wind chill using normal distn
# index = wdf[wdf['Wind_Chill(F)'].isna()].index
# value = np.random.normal(loc=wdf['Wind_Chill(F)'].mean(), scale=wdf['Wind_Chill(F)'].std(), size=wdf['Wind_Chill(F)'].isna().sum())
# wdf['Wind_Chill(F)'].fillna(pd.Series(value, index=index), inplace=True)
wdf['Wind_Chill(F)'] = wdf['Wind_Chill(F)'].fillna(wdf['Wind_Chill(F)'].mean())


# integrate missing data in wind speed using normal distn
# index = wdf[wdf['Wind_Speed(mph)'].isna()].index
# value = np.random.normal(loc=wdf['Wind_Speed(mph)'].mean(), scale=wdf['Wind_Speed(mph)'].std(), size=wdf['Wind_Speed(mph)'].isna().sum())
# wdf['Wind_Speed(mph)'].fillna(pd.Series(value, index=index), inplace=True)
wdf['Wind_Speed(mph)'] = wdf['Wind_Speed(mph)'].fillna(wdf['Wind_Speed(mph)'].mean())


# integrate missing data in Temperature ,Humidity  with mean

wdf['Temperature(F)'] = wdf['Temperature(F)'].fillna(wdf['Temperature(F)'].mean())
wdf['Humidity(%)'] = wdf['Humidity(%)'].fillna(wdf['Humidity(%)'].mean())
wdf['Visibility(mi)'] = wdf['Visibility(mi)'].fillna(wdf['Visibility(mi)'].mean())
wdf['Weather_Condition'] = wdf['Weather_Condition'].fillna('26')
wdf = wdf.drop_duplicates(subset=['timestamp'], keep='first')

wdf['Weather_Condition'].value_counts()
# wdf['Weather_Condition'] = wdf['Weather_Condition'].map(str)

# wdf['Weather_Condition'].isna().sum()
wdf.isna().sum()




The shape of the dataset is (6407, 16).




Weather_Condition    0
Wind_Chill(F)        0
Temperature(F)       0
Humidity(%)          0
Wind_Speed(mph)      0
Visibility(mi)       0
timestamp            0
dtype: int64

In [3]:
import xml.etree.ElementTree as ET

root = ET.parse('/kaggle/input/car-crashes-severity-prediction/holidays.xml').getroot()

df_cols = ["date","description"]
rows= []

for elem in root:
#     tag = {}
    s_row = elem.attrib.get('row')
    s_date= elem.find('date').text
    s_description= elem.find('description').text
    
    rows.append({"date":s_date,"description":s_description})
    
hdf = pd.DataFrame(rows,columns=df_cols)
hdf.head()


,date,description
0,2012-01-02,New Year Day
1,2012-01-16,Martin Luther King Jr. Day
2,2012-02-20,Presidents Day (Washingtons Birthday)
3,2012-05-28,Memorial Day
4,2012-07-04,Independence Day


In [4]:
# merge datafram with weather data
df['holiday']=df['timestamp'].str.contains("|".join(hdf['date']))
df['timestamp'] = df['timestamp'].map(str).apply(lambda x: x[0:13])



In [5]:



# wdf.info()
wdf['timestamp']=pd.to_datetime(wdf['timestamp'])
df['timestamp']=pd.to_datetime(df['timestamp'])


mdf = pd.merge(df, wdf, on='timestamp',how='left')
mdf.dropna(subset=['Visibility(mi)'], how='all', inplace=True)


We've got 6407 examples in the dataset with 14 featues, 1 ID, and the `Severity` of the crash.

By looking at the features and a sample from the data, the features look of numerical and catogerical types. What about some descriptive statistics?

In [6]:
mdf.drop(columns='ID').describe()

,Lat,Lng,Distance(mi),Severity,Wind_Chill(F),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi)
count,6407.000000,6407.000000,6407.000000,6407.000000,6407.000000,6407.00000,6407.000000,6407.000000,6407.000000
mean,37.765653,-122.405990,0.135189,2.293429,59.851594,59.93899,68.179669,10.864850,9.440442
std,0.032555,0.028275,0.396360,0.521225,6.549706,7.94523,16.163415,6.348389,1.647513
min,37.609619,-122.510440,0.000000,1.000000,31.100000,36.00000,10.000000,0.000000,0.120000
25%,37.737096,-122.412210,0.000000,2.000000,59.000000,54.00000,59.000000,6.000000,10.000000
50%,37.768238,-122.404835,0.000000,2.000000,59.762515,59.00000,70.000000,10.000000,10.000000
75%,37.787813,-122.392477,0.041000,3.000000,59.762515,64.90000,80.000000,15.000000,10.000000
max,37.825626,-122.349734,6.820000,4.000000,98.000000,98.00000,100.000000,40.300000,10.000000


The output shows desciptive statistics for the numerical features, `Lat`, `Lng`, `Distance(mi)`, and `Severity`. I'll use the numerical features to demonstrate how to train the model and make submissions. **However you shouldn't use the numerical features only to make the final submission if you want to make it to the top of the leaderboard.**

## Data Splitting

Now it's time to split the dataset for the training step. Typically the dataset is split into 3 subsets, namely, the training, validation and test sets. In our case, the test set is already predefined. So we'll split the "training" set into training and validation sets with 0.8:0.2 ratio. 

*Note: a good way to generate reproducible results is to set the seed to the algorithms that depends on randomization. This is done with the argument `random_state` in the following command* 

In [7]:
from sklearn.model_selection import train_test_split
# ,stratify=mdf[['Severity']]

train_df, val_df = train_test_split(mdf, test_size=0.2, random_state=42) # Try adding `,stratify=mdf` here

X_train = train_df.drop(columns=['ID', 'Severity'])
y_train = train_df['Severity']

X_val = val_df.drop(columns=['ID', 'Severity'])
y_val = val_df['Severity']

print(val_df)


        ID        Lat         Lng   Bump  Distance(mi)  Crossing  Give_Way  \
2488  2488  37.737483 -122.402230  False         0.000     False     False   
5479  5479  37.733130 -122.412227  False         0.000     False     False   
2399  2399  37.737400 -122.402270  False         0.407     False     False   
1330  1330  37.719284 -122.399544  False         0.000     False     False   
5921  5921  37.761803 -122.405869  False         0.010     False     False   
...    ...        ...         ...    ...           ...       ...       ...   
509    509  37.740032 -122.408096  False         0.000     False     False   
6221  6221  37.807710 -122.367640  False         0.037     False     False   
4578  4578  37.725469 -122.394187  False         0.106      True     False   
5437  5437  37.808110 -122.367190  False         0.037     False     False   
1732  1732  37.723620 -122.401340  False         0.136     False     False   

      Junction  No_Exit  Railway  ...  Side  Severity          

As pointed out eariler, I'll use the numerical features to train the classifier. **However, you shouldn't use the numerical features only to make the final submission if you want to make it to the top of the leaderboard.** 

In [8]:
# This cell is used to select the numerical features. IT SHOULD BE REMOVED AS YOU DO YOUR WORK.
X_train = X_train[['Lat', 'Lng','Weather_Condition','Temperature(F)','Humidity(%)','Wind_Chill(F)','Wind_Speed(mph)']]
X_val = X_val[['Lat', 'Lng','Weather_Condition','Temperature(F)','Humidity(%)','Wind_Chill(F)','Wind_Speed(mph)']]
print(X_val)

            Lat         Lng Weather_Condition  Temperature(F)  Humidity(%)  \
2488  37.737483 -122.402230               2.0            66.0         70.0   
5479  37.733130 -122.412227              11.0            71.0         53.0   
2399  37.737400 -122.402270               2.0            67.0         68.0   
1330  37.719284 -122.399544               1.0            58.0         62.0   
5921  37.761803 -122.405869               0.0            60.1         72.0   
...         ...         ...               ...             ...          ...   
509   37.740032 -122.408096               0.0            59.0         75.0   
6221  37.807710 -122.367640               1.0            62.1         70.0   
4578  37.725469 -122.394187               2.0            78.0         39.0   
5437  37.808110 -122.367190               9.0            64.0         80.0   
1732  37.723620 -122.401340               0.0            64.0         75.0   

      Wind_Chill(F)  Wind_Speed(mph)  
2488      66.000000     

## Model Training

Let's train a model with the data! We'll train a Random Forest Classifier to demonstrate the process of making submissions. 

In [9]:
from sklearn.ensemble import RandomForestClassifier

# Create an instance of the classifier
classifier = RandomForestClassifier( max_depth=2,random_state=0)

# Train the classifier
classifier = classifier.fit(X_train, y_train)

Now let's test our classifier on the validation dataset and see the accuracy.

In [10]:
# print(X_val)
# print(y_val)
print("The accuracy of the classifier on the validation set is ", (classifier.score(X_val, y_val)))

The accuracy of the classifier on the validation set is  0.7464898595943837


Well. That's a good start, right? A classifier that predicts all examples' `Severity` as 2 will get around 0.63. You should get better score as you add more features and do better data preprocessing.

## Submission File Generation

We have built a model and we'd like to submit our predictions on the test set! In order to do that, we'll load the test set, predict the class and save the submission file. 

First, we'll load the data.

In [11]:
test_df = pd.read_csv(os.path.join(dataset_path, 'test.csv'))
test_df.head()

test_df['holiday']=test_df['timestamp'].str.contains("|".join(hdf['date']))

test_df['timestamp'] = test_df['timestamp'].map(str).apply(lambda x: x[0:13])

In [12]:


test_df['timestamp']=pd.to_datetime(df['timestamp'])
test_mdf = pd.merge(test_df, wdf, on='timestamp',how="left")
print(len(test_mdf['ID']))
print(len(test_mdf['ID'].unique()))
test_mdf.isna().sum()
# wdf.dropna(subset=['Visibility(mi)'], how='all', inplace=True)
# wdf.dropna(subset=['Weather_Condition'], how='all', inplace=True)

1601
1601


ID                   0
Lat                  0
Lng                  0
Bump                 0
Distance(mi)         0
Crossing             0
Give_Way             0
Junction             0
No_Exit              0
Railway              0
Roundabout           0
Stop                 0
Amenity              0
Side                 0
timestamp            0
holiday              0
Weather_Condition    0
Wind_Chill(F)        0
Temperature(F)       0
Humidity(%)          0
Wind_Speed(mph)      0
Visibility(mi)       0
dtype: int64

Note that the test set has the same features and doesn't have the `Severity` column.
At this stage one must **NOT** forget to apply the same processing done on the training set on the features of the test set.

Now we'll add `Severity` column to the test `DataFrame` and add the values of the predicted class to it.

**I'll select the numerical features here as I did in the training set. DO NOT forget to change this step as you change the preprocessing of the training data.**

In [13]:
X_test = test_mdf.drop(columns=['ID'])



# You should update/remove the next line once you change the features used for training
X_test = X_test[['Lat', 'Lng','Weather_Condition','Temperature(F)','Humidity(%)','Wind_Chill(F)','Wind_Speed(mph)']]

y_test_predicted = classifier.predict(X_test)

test_mdf['Severity'] = y_test_predicted

test_mdf.head()

,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,...,Side,timestamp,holiday,Weather_Condition,Wind_Chill(F),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),Severity
0,6407,37.786060,-122.390900,False,0.039,False,False,True,False,False,...,R,2016-03-25 15:00:00,False,5.0,59.762515,64.0,58.0,23.0,10.0,2
1,6408,37.769609,-122.415057,False,0.202,False,False,False,False,False,...,R,2020-05-05 19:00:00,False,12.0,57.000000,57.0,83.0,22.0,10.0,2
2,6409,37.807495,-122.476021,False,0.000,False,False,False,False,False,...,R,2016-09-16 19:00:00,False,4.0,59.762515,62.1,80.0,9.2,10.0,2
3,6410,37.761818,-122.405869,False,0.000,False,False,True,False,False,...,R,2020-03-29 19:00:00,False,2.0,58.000000,58.0,70.0,10.0,10.0,2
4,6411,37.732350,-122.414100,False,0.670,False,False,False,False,False,...,R,2019-10-09 08:00:00,False,2.0,58.000000,58.0,65.0,3.0,10.0,2


Now we're ready to generate the submission file. The submission file needs the columns `ID` and `Severity` only.

In [14]:
# test_mdf[['ID', 'Severity']].duplicated()

test_mdf[['ID', 'Severity']].to_csv('/kaggle/working/submission.csv', index=False)

The remaining steps is to submit the generated file and are as follows. 

1. Press `Save Version` on the upper right corner of this notebook.
2. Write a `Version Name` of your choice and choose `Save & Run All (Commit)` then click `Save`.
3. Wait for the saved notebook to finish running the go to the saved notebook.
4. Scroll down until you see the output files then select the `submission.csv` file and click `Submit`.

Now your submission will be evaluated and your score will be updated on the leaderboard! CONGRATULATIONS!!

## Conclusion

In this notebook, we have demonstrated the essential steps that one should do in order to get "slightly" familiar with the data and the submission process. We chose not to go into details in each step to keep the welcoming notebook simple and make a room for improvement.

You're encourged to `Fork` the notebook, edit it, add your insights and use it to create your submission.